### Github Config

In [113]:
!mkdir -p ~/.ssh
!ssh-keyscan github.com >> ~/.ssh/known_hosts
!chmod 600 ~/.ssh/known_hosts

# github.com:22 SSH-2.0-0264bb16
# github.com:22 SSH-2.0-0264bb16
# github.com:22 SSH-2.0-0264bb16
# github.com:22 SSH-2.0-0264bb16
# github.com:22 SSH-2.0-0264bb16


In [115]:
!ls -al ~/.ssh

total 16
drwxr-xr-x 2 root root 4096 May 12 04:02 .
drwx------ 1 root root 4096 May 12 04:02 ..
-rw------- 1 root root  828 May 12 04:02 known_hosts


In [117]:
!ssh-keygen -t rsa -b 4096 -C "ivanjunqiwu99@gmail.com"

Generating public/private rsa key pair.
Enter file in which to save the key (/root/.ssh/id_rsa): 
Enter passphrase (empty for no passphrase): 
Enter same passphrase again: 
Your identification has been saved in /root/.ssh/id_rsa
Your public key has been saved in /root/.ssh/id_rsa.pub
The key fingerprint is:
SHA256:B1xn48yNr5H10+Qp15wHPG2wbl9brvEVObGict1+sbs ivanjunqiwu99@gmail.com
The key's randomart image is:
+---[RSA 4096]----+
|          . + .  |
|       . . * = + |
|        o   = B.+|
|         .   = OO|
|        S . oo+XX|
|         .  o+=+O|
|         . o...o*|
|          o   .=o|
|              .E=|
+----[SHA256]-----+


In [118]:
!cat ~/.ssh/id_rsa.pub

ssh-rsa AAAAB3NzaC1yc2EAAAADAQABAAACAQDpFlKedfuUfgtOU+Dil90kTyS+gks0M3yTJ5I1o1vhbSWp6N1o1m0coaVLrEI3HlhLYRme2s531y4yU5bfhXpUuPLW1up0rurvIyFm+aprr6TlKkivtuTfwGdCaMW7dLeTkYew6uxmjxL4AMeGpLYILbqGOQKIru/6TpbDyNY7n3MmrI7w1OdCnGFhIJMZfZ7z/bUNms3/A6DmW9452mE6OKZ1WWepPXhmGZZa3eNDKs3v1Wxo8OiMSMBm4xg2+Ifg8vGVyM2h/Mflc+pHzGiHMFswGuLeSAnye2FYNW2DvFiHjAdJrIpayZ88Wx5wNEUU9VIZSiMuy3e8MlegYR0lQ6Ki8nPA/x4TzsxpexVoFgN2OH1LN4F18Dt2vlPiNRlScCh+7Mqfth1Dp+2154ov4NkLpE+s1a4BIE+a61M0t5SD1zuRxqeBkNOt8ywCveSDyY8VeYGu02PhvHTBwCQOnN/0FlOtlqMo72uPNDqMCGzqeHPDL+GCJMDFukPNvz0UAB0WBV8s7gHZw/+mbpYJofvhcvE2ABD+tTdqy6PgEKbJD9LntgqnpfPEmCa0v4hpQ46NR0vdazbQd5gQPaV49wknbkm9uUF2T2pc7AAX6Rpwj52iw/eweSM9+e4A6CYswnGji5TATtUXT52oaH+ltf4i0Qa12+oOUSbR/DpbYw== ivanjunqiwu99@gmail.com


In [119]:
!git clone git@github.com:ivanckng/CS610_AML.git

Cloning into 'CS610_AML'...
remote: Enumerating objects: 56, done.
remote: Counting objects: 100% (56/56), done.
remote: Compressing objects: 100% (46/46), done.
remote: Total 56 (delta 25), reused 30 (delta 9), pack-reused 0 (from 0)
Receiving objects: 100% (56/56), 5.06 MiB | 25.02 MiB/s, done.
Resolving deltas: 100% (25/25), done.


### Import Libraries

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [28]:
!pip install xgboost
!pip install lightgbm

In [29]:
import pandas as pd
from sklearn import linear_model, metrics, preprocessing, tree, model_selection, feature_selection, ensemble
import numpy as np
import matplotlib.pyplot as plt
import xgboost as xgb
from collections import Counter
import lightgbm as lgb

import nltk
import string
import re
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.sentiment.vader import SentimentIntensityAnalyzer

In [10]:
try:
    stopwords.words('english')
except LookupError:
    nltk.download('stopwords')
try:
    WordNetLemmatizer().lemmatize('cats')
except LookupError:
    nltk.download('wordnet')
    nltk.download('omw-1.4')
try:
    SentimentIntensityAnalyzer().polarity_scores("hello")
except LookupError:
    nltk.download('vader_lexicon')

### Loading training and tesing data

In [51]:
train_set = pd.read_csv('/content/drive/MyDrive/10 Github/CS610_AML/01 Assignment 1/train.csv')
test_set = pd.read_csv('/content/drive/MyDrive/10 Github/CS610_AML/01 Assignment 1/test.csv')

### Data Processing & EDA

In [52]:
train_set.head()

,created_at,default_profile,default_profile_image,description,favourites_count,followers_count,friends_count,geo_enabled,id,lang,location,profile_background_image_url,profile_image_url,screen_name,statuses_count,verified,average_tweets_per_day,account_age_days,target
0,2012-01-15 23:40:09,True,False,Cosplayer/Fitness lover. Come to me https://t....,74,7,0,False,465096524,en,unknown,http://abs.twimg.com/images/themes/theme1/bg.png,http://pbs.twimg.com/profile_images/9666745212...,reml5477,20,False,0.006,3138,1
1,2016-10-04 00:44:39,False,False,pobody’s nerfect,50443,164,590,True,783105517673648132,cy,she/her,http://abs.twimg.com/images/themes/theme1/bg.png,http://pbs.twimg.com/profile_images/1281752126...,kinlibra,6469,False,4.572,1415,0
2,2009-05-23 04:04:13,False,False,gracias por participar 🏅,9394,208,189,False,41970759,es,La diaspora,http://abs.twimg.com/images/themes/theme17/bg.gif,http://pbs.twimg.com/profile_images/1233811596...,_delaualau,30296,False,7.378,4106,0
3,2009-05-17 04:31:31,False,False,Stand Up Comedian/Actor from North Philadelphi...,46,66180,1090,True,40607946,en,"Calabasas, CA",http://abs.twimg.com/images/themes/theme1/bg.png,http://pbs.twimg.com/profile_images/1184851104...,SpankHorton,164957,False,40.116,4112,0
4,2009-02-16 13:11:21,True,False,Assignment Editor at NBC10 and President of Ja...,1223,487,867,True,20983433,en,"Jenkintown, PA",http://abs.twimg.com/images/themes/theme1/bg.png,http://pbs.twimg.com/profile_images/5234863934...,javelinjt,1752,False,0.417,4201,0


In [53]:
test_set.head()

,index,created_at,default_profile,default_profile_image,description,favourites_count,followers_count,friends_count,geo_enabled,id,lang,location,profile_background_image_url,profile_image_url,screen_name,statuses_count,verified,average_tweets_per_day,account_age_days
0,0,2007-03-27 22:56:04,False,False,Dad. #ADHD @fasternormal Pod host. Author/keyn...,22290,163813,1746,True,2569261,en,"New York, NY",http://abs.twimg.com/images/themes/theme1/bg.png,http://pbs.twimg.com/profile_images/1186468889...,petershankman,42013,True,8.586,4893
1,1,2013-03-30 08:47:01,True,False,NaN,685,44,637,False,1316038452,NaN,unknown,http://abs.twimg.com/images/themes/theme1/bg.png,http://pbs.twimg.com/profile_images/9637876928...,Mihail_HD,111,False,0.041,2699
2,2,2009-04-25 15:24:08,False,False,#UGAMOM #GODAWGS,12257,276,194,True,35233977,de,"Augusta, GA",http://abs.twimg.com/images/themes/theme3/bg.gif,http://pbs.twimg.com/profile_images/1246470377...,TinkerMazell,18142,False,4.390,4133
3,3,2010-08-06 22:21:56,True,False,"golfer, traveler, car guy, PCA member, owns 19...",30806,1124,4999,True,175542625,en,New York,http://abs.twimg.com/images/themes/theme1/bg.png,http://pbs.twimg.com/profile_images/1391357162...,joecangolfICC,20624,False,5.627,3665
4,4,2015-03-02 12:29:39,True,False,Do you like it gently? Come in! 💚💚💚 https://t....,224,22,265,False,3056068536,en,unknown,http://abs.twimg.com/images/themes/theme1/bg.png,http://pbs.twimg.com/profile_images/9680856906...,TiesiogDovydas,64,False,0.032,1996


In [54]:
print("Size of Training Instances:", train_set.shape[0])
print("Sice of Features:", train_set.shape[1])
print(train_set.info())

print("Size of Testing Instances:", test_set.shape[0])
print("Sice of Features:", test_set.shape[1])
print(test_set.info())

Size of Training Instances: 26206
Sice of Features: 19
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26206 entries, 0 to 26205
Data columns (total 19 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   created_at                    26206 non-null  object 
 1   default_profile               26206 non-null  bool   
 2   default_profile_image         26206 non-null  bool   
 3   description                   21115 non-null  object 
 4   favourites_count              26206 non-null  int64  
 5   followers_count               26206 non-null  int64  
 6   friends_count                 26206 non-null  int64  
 7   geo_enabled                   26206 non-null  bool   
 8   id                            26206 non-null  int64  
 9   lang                          20618 non-null  object 
 10  location                      26204 non-null  object 
 11  profile_background_image_url  22971 non-null  object 
 12  profi

In [55]:
train_set.drop(columns=['created_at','id', 'profile_background_image_url', 'profile_image_url', 'screen_name', 'location'], inplace=True)
test_set.drop(columns=['created_at','id', 'profile_background_image_url', 'profile_image_url', 'screen_name', 'location'], inplace=True)

In [56]:
# train_set['lang'] = train_set['lang'].isna().astype(int)
# train_set['lang'] = np.where(train_set['lang'] == 1, 0, 1)
train_set['lang'] = train_set['lang'].fillna('unknown')
# test_set['lang'] = test_set['lang'].isna().astype(int)
# test_set['lang'] = np.where(test_set['lang'] == 1, 0, 1)
test_set['lang'] = test_set['lang'].fillna('unknown')

In [57]:
train_set['default_profile'] = train_set['default_profile'].astype(int)
train_set['default_profile_image'] = train_set['default_profile_image'].astype(int)
train_set['geo_enabled'] = train_set['geo_enabled'].astype(int)
train_set['verified'] = train_set['verified'].astype(int)

test_set['default_profile'] = test_set['default_profile'].astype(int)
test_set['default_profile_image'] = test_set['default_profile_image'].astype(int)
test_set['geo_enabled'] = test_set['geo_enabled'].astype(int)
test_set['verified'] = test_set['verified'].astype(int)

In [58]:
train_set['lang'] = train_set['lang'].astype('category')
test_set['lang'] = test_set['lang'].astype('category')

### NLP

In [60]:
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

def preprocess_text(text):
    if pd.isna(text) or text == "":
        return ""
    text = text.lower()
    text = text.translate(str.maketrans('', '', string.punctuation))
    text = re.sub(r'\d+', '', text)
    words = text.split()
    words = [lemmatizer.lemmatize(word) for word in words if word not in stop_words]
    return " ".join(words)

train_set['description'] = train_set['description'].fillna('')
train_set['description'] = train_set['description'].apply(preprocess_text)
test_set['description'] = test_set['description'].fillna('')
test_set['description'] = test_set['description'].apply(preprocess_text)

In [61]:
train_set['desc_len'] = train_set['description'].apply(len)
train_set['desc_wordcount'] = train_set['description'].apply(lambda x: len(x.split()))


test_set['desc_len'] = test_set['description'].apply(len)
test_set['desc_wordcount'] = test_set['description'].apply(lambda x: len(x.split()))

In [62]:
sid = SentimentIntensityAnalyzer()
def get_sentiment_scores(text):
    if pd.isna(text) or text == "":
        return 0.0
    return sid.polarity_scores(text)['compound']

train_set['desc_sentiment'] = train_set['description'].apply(get_sentiment_scores)
test_set['desc_sentiment'] = test_set['description'].apply(get_sentiment_scores)

#### Data Split

In [63]:
train_set = train_set.drop(columns=['description'])
test_set = test_set.drop(columns=['description'])

In [64]:
num_majority = train_set[train_set['target'] == 0].shape[0]
num_minority = train_set[train_set['target'] == 1].shape[0]

In [65]:
x_train, x_test, y_train, y_test = model_selection.train_test_split(train_set.drop(columns=['target']), train_set['target'], test_size=0.2, shuffle = True, random_state=42, stratify=train_set['target'])

In [66]:
counter = Counter(y_train)
estimate_scale_pos_weight = counter[0] / counter[1]

In [67]:
train_predictors = train_set.drop(columns=['target'])
train_target = train_set['target']

In [68]:
counter_train = Counter(train_target)
estimate_scale_pos_weight_train = counter_train[0] / counter_train[1]

In [69]:
x_train

,default_profile,default_profile_image,favourites_count,followers_count,friends_count,geo_enabled,lang,statuses_count,verified,average_tweets_per_day,account_age_days,desc_len,desc_wordcount,desc_sentiment
16919,0,0,1534,45,222,0,pt,2759,0,0.705,3915,80,18,0.0000
3662,0,0,2207,11557,820,1,en,7577,0,1.984,3820,33,6,0.0000
6669,0,0,738,36684,185,1,ar,141155,0,42.414,3328,46,6,0.0000
7412,0,0,24553,970,968,1,en,10305,0,2.783,3703,27,3,0.0000
8506,0,0,8942,1898,807,1,en,10848,1,2.601,4170,64,9,0.0000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17356,0,0,2768,295,295,1,en,13869,0,4.086,3394,13,3,0.0762
92,0,0,63616,883,742,1,it,52246,0,25.954,2013,20,4,0.0000
8298,0,0,3256,341,186,1,en,1580,0,0.386,4092,66,12,0.4215
14834,1,0,1332,8,31,0,en,1333,0,0.912,1462,21,2,0.0000


### Random Forest

In [ ]:
rf_model = ensemble.RandomForestClassifier(
    random_state=42,
    class_weight='balanced_subsample',
)

In [ ]:
param_grid_rf = {
    'n_estimators': [100, 200, 300, 400, 500],
    'criterion': ['gini', 'entropy'],
    'max_features': ['sqrt', 'log2']
}

In [ ]:
kfold = model_selection.StratifiedKFold(n_splits=10, shuffle=True, random_state=42)

In [ ]:
grid_search_rf = model_selection.GridSearchCV(estimator=rf_model,
                              param_grid=param_grid_rf,
                              cv=kfold,
                              scoring='roc_auc',
                              n_jobs=-1,
                              verbose=2
                             )

In [ ]:
grid_search_rf.fit(x_train, y_train)

Fitting 10 folds for each of 20 candidates, totalling 200 fits


GridSearchCV(cv=StratifiedKFold(n_splits=10, random_state=42, shuffle=True),
             estimator=RandomForestClassifier(class_weight='balanced_subsample',
                                              random_state=42),
             n_jobs=-1,
             param_grid={'criterion': ['gini', 'entropy'],
                         'max_features': ['sqrt', 'log2'],
                         'n_estimators': [100, 200, 300, 400, 500]},
             scoring='roc_auc', verbose=2)

In [ ]:
best_rf_model = grid_search_rf.best_estimator_

In [ ]:
y_train_pred_proba_rf = best_rf_model.predict_proba(x_train)[:, 1]
train_auc_rf = metrics.roc_auc_score(y_train, y_train_pred_proba_rf)
print(f"Train AUC for tuned Random Forest: {train_auc_rf:.4f}")
y_test_pred_proba_rf = best_rf_model.predict_proba(x_test)[:, 1]
test_auc_rf = metrics.roc_auc_score(y_test, y_test_pred_proba_rf)
print(f"Test AUC for tuned Random Forest: {test_auc_rf:.4f}")

Train AUC for tuned Random Forest: 1.0000
Test AUC for tuned Random Forest: 0.9413


In [ ]:
train_predictors = train_set.drop(columns=['target'])
train_target = train_set['target']

In [ ]:
grid_search_rf.fit(train_predictors, train_target)

Fitting 10 folds for each of 20 candidates, totalling 200 fits


GridSearchCV(cv=StratifiedKFold(n_splits=10, random_state=42, shuffle=True),
             estimator=RandomForestClassifier(class_weight='balanced_subsample',
                                              random_state=42),
             n_jobs=-1,
             param_grid={'criterion': ['gini', 'entropy'],
                         'max_features': ['sqrt', 'log2'],
                         'n_estimators': [100, 200, 300, 400, 500]},
             scoring='roc_auc', verbose=2)

In [ ]:
best_rf_model = grid_search_rf.best_estimator_

In [ ]:
train_target_pred_proba_rf = best_rf_model.predict_proba(train_predictors)[:, 1]
train_auc_rf = metrics.roc_auc_score(train_target, train_target_pred_proba_rf)
print(f"Train AUC for tuned Random Forest: {train_auc_rf:.4f}")

Train AUC for tuned Random Forest: 1.0000


In [ ]:
test_set_predictors = test_set.drop(columns=['index'])
test_set_target_proba = best_rf_model.predict_proba(test_set_predictors)[:, 1]

test_set_prob = pd.DataFrame({'index': test_set['index'], 'target': test_set_target_proba})
test_set_prob.to_csv('rf_test_set_prob.csv', index=False)

### XGBoost

In [ ]:
xgb_model = xgb.XGBClassifier(use_label_encoder=False,
eval_metric='logloss',
random_state=42,
tree_method='hist',
device='cuda',
scale_pos_weight=estimate_scale_pos_weight,
enable_categorical = True
)  # Important:  Set eval_metric

In [ ]:
param_grid_xgb = {
    'n_estimators': [500,600,700,800,900,1000],
    'learning_rate': [0.001, 0.01, 0.02]
}

In [ ]:
grid_search_xgb = model_selection.GridSearchCV(
    estimator=xgb_model,
    param_grid=param_grid_xgb,
    scoring='roc_auc',
    cv=10,
    n_jobs=-1,
    verbose=1
)

In [ ]:
grid_search_xgb.fit(x_train, y_train)

Fitting 10 folds for each of 18 candidates, totalling 180 fits


/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [12:29:34] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


GridSearchCV(cv=10,
             estimator=XGBClassifier(base_score=None, booster=None,
                                     callbacks=None, colsample_bylevel=None,
                                     colsample_bynode=None,
                                     colsample_bytree=None, device='cuda',
                                     early_stopping_rounds=None,
                                     enable_categorical=True,
                                     eval_metric='logloss', feature_types=None,
                                     gamma=None, grow_policy=None,
                                     importance_type=None,
                                     interaction_constraints=None,
                                     learning_rat...
                                     max_cat_to_onehot=None,
                                     max_delta_step=None, max_depth=None,
                                     max_leaves=None, min_child_weight=None,
                                     missing=nan, monotone_constraints=None,
                                     multi_strategy=None, n_estimators=None,
                                     n_jobs=None, num_parallel_tree=None,
                                     random_state=42, ...),
             n_jobs=-1,
             param_grid={'learning_rate': [0.001, 0.01, 0.02],
                         'n_estimators': [500, 600, 700, 800, 900, 1000]},
             scoring='roc_auc', verbose=1)

In [ ]:
best_xgb_model = grid_search_xgb.best_estimator_
y_pred_train_proba_xgb = best_xgb_model.predict_proba(x_train)[:, 1]
train_auc_xgb = metrics.roc_auc_score(y_train, y_pred_train_proba_xgb)
print(f"Train AUC for tuned XGBoost: {train_auc_xgb:.4f}")
y_pred_test_proba_xgb = best_xgb_model.predict_proba(x_test)[:, 1]
test_auc_xgb = metrics.roc_auc_score(y_test, y_pred_test_proba_xgb)
print(f"Test AUC for tuned XGBoost: {test_auc_xgb:.4f}")

Train AUC for tuned XGBoost: 0.9811
Test AUC for tuned XGBoost: 0.9406


In [ ]:
# Original XGBoost model
grid_search_xgb.fit(train_predictors, train_target)

Fitting 10 folds for each of 18 candidates, totalling 180 fits


/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [12:37:53] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


GridSearchCV(cv=10,
             estimator=XGBClassifier(base_score=None, booster=None,
                                     callbacks=None, colsample_bylevel=None,
                                     colsample_bynode=None,
                                     colsample_bytree=None, device='cuda',
                                     early_stopping_rounds=None,
                                     enable_categorical=True,
                                     eval_metric='logloss', feature_types=None,
                                     gamma=None, grow_policy=None,
                                     importance_type=None,
                                     interaction_constraints=None,
                                     learning_rat...
                                     max_cat_to_onehot=None,
                                     max_delta_step=None, max_depth=None,
                                     max_leaves=None, min_child_weight=None,
                                     missing=nan, monotone_constraints=None,
                                     multi_strategy=None, n_estimators=None,
                                     n_jobs=None, num_parallel_tree=None,
                                     random_state=42, ...),
             n_jobs=-1,
             param_grid={'learning_rate': [0.001, 0.01, 0.02],
                         'n_estimators': [500, 600, 700, 800, 900, 1000]},
             scoring='roc_auc', verbose=1)

In [ ]:
best_xgb_model = grid_search_xgb.best_estimator_
train_target_proba = best_xgb_model.predict_proba(train_predictors)[:, 1]
train_auc_xgb = metrics.roc_auc_score(train_target, train_target_proba)
print(f"Train AUC for tuned XGBoost: {train_auc_xgb:.4f}")

Train AUC for tuned XGBoost: 0.9764


In [ ]:
test_set_predictors = test_set.drop(columns=['index'])
test_set_target_proba = best_xgb_model.predict_proba(test_set_predictors)[:, 1]

test_set_prob = pd.DataFrame({'index': test_set['index'], 'target': test_set_target_proba})
test_set_prob.to_csv('/content/drive/MyDrive/10 Github/CS610_AML/xgb_test_set_prob.csv', index=False)

In [ ]:
best_xgb_model.feature_importances_

array([0.03113176, 0.0092409 , 0.06154347, 0.17197716, 0.04745121,
       0.07212621, 0.00968822, 0.03960389, 0.40006047, 0.05895726,
       0.0436013 , 0.01872987, 0.02009667, 0.01579155], dtype=float32)

### LGB

In [128]:
lgbm_classifier = lgb.LGBMClassifier(
    objective='binary',
    metric='auc',
    n_estimators=500,
    learning_rate=0.1,
    num_leaves=31,
    max_depth=-1,
    min_child_samples=20,
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=42,
    n_jobs=-1,
    device='gpu',
    categorical_feature=['lang'],
    scale_pos_weight=estimate_scale_pos_weight,
)

In [129]:
cv_strategy = model_selection.StratifiedKFold(n_splits=10, shuffle=True, random_state=42)

In [130]:
param_grid_lgbm = {
    'n_estimators': [400, 500, 600],
    'learning_rate': [0.001, 0.01, 0.02],
    'num_leaves': [15, 30, 45]

}

In [131]:
cv_strategy = model_selection.StratifiedKFold(n_splits=10, shuffle=True, random_state=42) # CV折数可以调整，3折会快一些

In [132]:
grid_search_lgb = model_selection.GridSearchCV(
    estimator=lgbm_classifier,
    param_grid=param_grid_lgbm,
    scoring='roc_auc',
    cv=cv_strategy,
    n_jobs=1,
    verbose=2
)

In [133]:
grid_search_lgb.fit(x_train,y_train)

Fitting 10 folds for each of 27 candidates, totalling 270 fits
[LightGBM] [Warning] categorical_feature is set=lang, categorical_column=6 will be ignored. Current value: categorical_feature=lang
[LightGBM] [Info] Number of positive: 6323, number of negative: 12544
[LightGBM] [Info] This is the GPU trainer!!
[LightGBM] [Info] Total Bins 1998
[LightGBM] [Info] Number of data points in the train set: 18867, number of used features: 14
[LightGBM] [Info] Using GPU Device: Tesla T4, Vendor: NVIDIA Corporation
[LightGBM] [Info] Compiling OpenCL Kernel with 256 bins...
[LightGBM] [Info] GPU programs have been built
[LightGBM] [Info] Size of histogram bin entry: 8
[LightGBM] [Info] 11 dense feature groups (0.22 MB) transferred to GPU in 0.000860 secs. 1 sparse feature groups
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.335135 -> initscore=-0.685049
[LightGBM] [Info] Start training from score -0.685049


/usr/local/lib/python3.11/dist-packages/lightgbm/basic.py:2118: UserWarning: categorical_feature keyword has been found in `params` and will be ignored.
Please use categorical_feature argument of the Dataset constructor to pass this parameter.
  _log_warning(
/usr/local/lib/python3.11/dist-packages/lightgbm/basic.py:2140: UserWarning: categorical_feature in param dict is overridden.
  _log_warning(f"{cat_alias} in param dict is overridden.")


[CV] END learning_rate=0.001, n_estimators=400, num_leaves=15; total time=   1.1s
[LightGBM] [Warning] categorical_feature is set=lang, categorical_column=6 will be ignored. Current value: categorical_feature=lang
[LightGBM] [Info] Number of positive: 6323, number of negative: 12544
[LightGBM] [Info] This is the GPU trainer!!
[LightGBM] [Info] Total Bins 1998
[LightGBM] [Info] Number of data points in the train set: 18867, number of used features: 14
[LightGBM] [Info] Using GPU Device: Tesla T4, Vendor: NVIDIA Corporation
[LightGBM] [Info] Compiling OpenCL Kernel with 256 bins...
[LightGBM] [Info] GPU programs have been built
[LightGBM] [Info] Size of histogram bin entry: 8
[LightGBM] [Info] 11 dense feature groups (0.22 MB) transferred to GPU in 0.000862 secs. 1 sparse feature groups
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.335135 -> initscore=-0.685049
[LightGBM] [Info] Start training from score -0.685049


/usr/local/lib/python3.11/dist-packages/lightgbm/basic.py:2118: UserWarning: categorical_feature keyword has been found in `params` and will be ignored.
Please use categorical_feature argument of the Dataset constructor to pass this parameter.
  _log_warning(
/usr/local/lib/python3.11/dist-packages/lightgbm/basic.py:2140: UserWarning: categorical_feature in param dict is overridden.
  _log_warning(f"{cat_alias} in param dict is overridden.")


[CV] END learning_rate=0.001, n_estimators=400, num_leaves=15; total time=   1.1s
[LightGBM] [Warning] categorical_feature is set=lang, categorical_column=6 will be ignored. Current value: categorical_feature=lang
[LightGBM] [Info] Number of positive: 6323, number of negative: 12544
[LightGBM] [Info] This is the GPU trainer!!
[LightGBM] [Info] Total Bins 1998
[LightGBM] [Info] Number of data points in the train set: 18867, number of used features: 14
[LightGBM] [Info] Using GPU Device: Tesla T4, Vendor: NVIDIA Corporation
[LightGBM] [Info] Compiling OpenCL Kernel with 256 bins...
[LightGBM] [Info] GPU programs have been built
[LightGBM] [Info] Size of histogram bin entry: 8
[LightGBM] [Info] 11 dense feature groups (0.22 MB) transferred to GPU in 0.000921 secs. 1 sparse feature groups
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.335135 -> initscore=-0.685049
[LightGBM] [Info] Start training from score -0.685049


...

[CV] END learning_rate=0.02, n_estimators=600, num_leaves=45; total time=   3.1s
[LightGBM] [Warning] categorical_feature is set=lang, categorical_column=6 will be ignored. Current value: categorical_feature=lang
[LightGBM] [Info] Number of positive: 6324, number of negative: 12544
[LightGBM] [Info] This is the GPU trainer!!
[LightGBM] [Info] Total Bins 2004
[LightGBM] [Info] Number of data points in the train set: 18868, number of used features: 14
[LightGBM] [Info] Using GPU Device: Tesla T4, Vendor: NVIDIA Corporation
[LightGBM] [Info] Compiling OpenCL Kernel with 256 bins...
[LightGBM] [Info] GPU programs have been built
[LightGBM] [Info] Size of histogram bin entry: 8
[LightGBM] [Info] 11 dense feature groups (0.22 MB) transferred to GPU in 0.000897 secs. 1 sparse feature groups
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.335171 -> initscore=-0.684891
[LightGBM] [Info] Start training from score -0.684891


/usr/local/lib/python3.11/dist-packages/lightgbm/basic.py:2118: UserWarning: categorical_feature keyword has been found in `params` and will be ignored.
Please use categorical_feature argument of the Dataset constructor to pass this parameter.
  _log_warning(
/usr/local/lib/python3.11/dist-packages/lightgbm/basic.py:2140: UserWarning: categorical_feature in param dict is overridden.
  _log_warning(f"{cat_alias} in param dict is overridden.")


[CV] END learning_rate=0.02, n_estimators=600, num_leaves=45; total time=   3.1s
[LightGBM] [Warning] categorical_feature is set=lang, categorical_column=6 will be ignored. Current value: categorical_feature=lang
[LightGBM] [Info] Number of positive: 7026, number of negative: 13938
[LightGBM] [Info] This is the GPU trainer!!
[LightGBM] [Info] Total Bins 2004
[LightGBM] [Info] Number of data points in the train set: 20964, number of used features: 14
[LightGBM] [Info] Using GPU Device: Tesla T4, Vendor: NVIDIA Corporation
[LightGBM] [Info] Compiling OpenCL Kernel with 256 bins...
[LightGBM] [Info] GPU programs have been built
[LightGBM] [Info] Size of histogram bin entry: 8
[LightGBM] [Info] 11 dense feature groups (0.24 MB) transferred to GPU in 0.000989 secs. 1 sparse feature groups
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.335146 -> initscore=-0.685001
[LightGBM] [Info] Start training from score -0.685001


/usr/local/lib/python3.11/dist-packages/lightgbm/basic.py:2118: UserWarning: categorical_feature keyword has been found in `params` and will be ignored.
Please use categorical_feature argument of the Dataset constructor to pass this parameter.
  _log_warning(
/usr/local/lib/python3.11/dist-packages/lightgbm/basic.py:2140: UserWarning: categorical_feature in param dict is overridden.
  _log_warning(f"{cat_alias} in param dict is overridden.")


GridSearchCV(cv=StratifiedKFold(n_splits=10, random_state=42, shuffle=True),
             estimator=LGBMClassifier(categorical_feature=['lang'],
                                      colsample_bytree=0.8, device='gpu',
                                      metric='auc', n_estimators=500, n_jobs=-1,
                                      objective='binary', random_state=42,
                                      scale_pos_weight=1.9837745516652434,
                                      subsample=0.8),
             n_jobs=1,
             param_grid={'learning_rate': [0.001, 0.01, 0.02],
                         'n_estimators': [400, 500, 600],
                         'num_leaves': [15, 30, 45]},
             scoring='roc_auc', verbose=2)

In [134]:
best_lgb_model = grid_search_lgb.best_estimator_
y_pred_train_proba_lgb= best_lgb_model.predict_proba(x_train)[:, 1]
train_auc_lgb = metrics.roc_auc_score(y_train, y_pred_train_proba_lgb)
print(f"Train AUC for tuned XGBoost: {train_auc_lgb:.4f}")
y_pred_test_proba_lgb = best_lgb_model.predict_proba(x_test)[:, 1]
test_auc_lgb = metrics.roc_auc_score(y_test, y_pred_test_proba_lgb)
print(f"Test AUC for tuned XGBoost: {test_auc_lgb:.4f}")


Train AUC for tuned XGBoost: 0.9797
Test AUC for tuned XGBoost: 0.9440


In [135]:
grid_search_lgb.fit(train_predictors, train_target)

Fitting 10 folds for each of 27 candidates, totalling 270 fits
[LightGBM] [Warning] categorical_feature is set=lang, categorical_column=6 will be ignored. Current value: categorical_feature=lang
[LightGBM] [Info] Number of positive: 7904, number of negative: 15681
[LightGBM] [Info] This is the GPU trainer!!
[LightGBM] [Info] Total Bins 2002
[LightGBM] [Info] Number of data points in the train set: 23585, number of used features: 14
[LightGBM] [Info] Using GPU Device: Tesla T4, Vendor: NVIDIA Corporation
[LightGBM] [Info] Compiling OpenCL Kernel with 256 bins...
[LightGBM] [Info] GPU programs have been built
[LightGBM] [Info] Size of histogram bin entry: 8
[LightGBM] [Info] 11 dense feature groups (0.27 MB) transferred to GPU in 0.001092 secs. 1 sparse feature groups
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.335128 -> initscore=-0.685081
[LightGBM] [Info] Start training from score -0.685081


/usr/local/lib/python3.11/dist-packages/lightgbm/basic.py:2118: UserWarning: categorical_feature keyword has been found in `params` and will be ignored.
Please use categorical_feature argument of the Dataset constructor to pass this parameter.
  _log_warning(
/usr/local/lib/python3.11/dist-packages/lightgbm/basic.py:2140: UserWarning: categorical_feature in param dict is overridden.
  _log_warning(f"{cat_alias} in param dict is overridden.")


[CV] END learning_rate=0.001, n_estimators=400, num_leaves=15; total time=   1.1s
[LightGBM] [Warning] categorical_feature is set=lang, categorical_column=6 will be ignored. Current value: categorical_feature=lang
[LightGBM] [Info] Number of positive: 7904, number of negative: 15681
[LightGBM] [Info] This is the GPU trainer!!
[LightGBM] [Info] Total Bins 2011
[LightGBM] [Info] Number of data points in the train set: 23585, number of used features: 14
[LightGBM] [Info] Using GPU Device: Tesla T4, Vendor: NVIDIA Corporation
[LightGBM] [Info] Compiling OpenCL Kernel with 256 bins...
[LightGBM] [Info] GPU programs have been built
[LightGBM] [Info] Size of histogram bin entry: 8
[LightGBM] [Info] 11 dense feature groups (0.27 MB) transferred to GPU in 0.001022 secs. 1 sparse feature groups
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.335128 -> initscore=-0.685081
[LightGBM] [Info] Start training from score -0.685081


/usr/local/lib/python3.11/dist-packages/lightgbm/basic.py:2118: UserWarning: categorical_feature keyword has been found in `params` and will be ignored.
Please use categorical_feature argument of the Dataset constructor to pass this parameter.
  _log_warning(
/usr/local/lib/python3.11/dist-packages/lightgbm/basic.py:2140: UserWarning: categorical_feature in param dict is overridden.
  _log_warning(f"{cat_alias} in param dict is overridden.")


[CV] END learning_rate=0.001, n_estimators=400, num_leaves=15; total time=   1.1s
[LightGBM] [Warning] categorical_feature is set=lang, categorical_column=6 will be ignored. Current value: categorical_feature=lang
[LightGBM] [Info] Number of positive: 7904, number of negative: 15681
[LightGBM] [Info] This is the GPU trainer!!
[LightGBM] [Info] Total Bins 2009
[LightGBM] [Info] Number of data points in the train set: 23585, number of used features: 14
[LightGBM] [Info] Using GPU Device: Tesla T4, Vendor: NVIDIA Corporation
[LightGBM] [Info] Compiling OpenCL Kernel with 256 bins...
[LightGBM] [Info] GPU programs have been built
[LightGBM] [Info] Size of histogram bin entry: 8
[LightGBM] [Info] 11 dense feature groups (0.27 MB) transferred to GPU in 0.000997 secs. 1 sparse feature groups
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.335128 -> initscore=-0.685081
[LightGBM] [Info] Start training from score -0.685081


...

[CV] END learning_rate=0.02, n_estimators=600, num_leaves=45; total time=   3.2s
[LightGBM] [Warning] categorical_feature is set=lang, categorical_column=6 will be ignored. Current value: categorical_feature=lang
[LightGBM] [Info] Number of positive: 7905, number of negative: 15681
[LightGBM] [Info] This is the GPU trainer!!
[LightGBM] [Info] Total Bins 2006
[LightGBM] [Info] Number of data points in the train set: 23586, number of used features: 14
[LightGBM] [Info] Using GPU Device: Tesla T4, Vendor: NVIDIA Corporation
[LightGBM] [Info] Compiling OpenCL Kernel with 256 bins...
[LightGBM] [Info] GPU programs have been built
[LightGBM] [Info] Size of histogram bin entry: 8
[LightGBM] [Info] 11 dense feature groups (0.27 MB) transferred to GPU in 0.000933 secs. 1 sparse feature groups
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.335156 -> initscore=-0.684954
[LightGBM] [Info] Start training from score -0.684954


/usr/local/lib/python3.11/dist-packages/lightgbm/basic.py:2118: UserWarning: categorical_feature keyword has been found in `params` and will be ignored.
Please use categorical_feature argument of the Dataset constructor to pass this parameter.
  _log_warning(
/usr/local/lib/python3.11/dist-packages/lightgbm/basic.py:2140: UserWarning: categorical_feature in param dict is overridden.
  _log_warning(f"{cat_alias} in param dict is overridden.")


[CV] END learning_rate=0.02, n_estimators=600, num_leaves=45; total time=   3.2s
[LightGBM] [Warning] categorical_feature is set=lang, categorical_column=6 will be ignored. Current value: categorical_feature=lang
[LightGBM] [Info] Number of positive: 8783, number of negative: 17423
[LightGBM] [Info] This is the GPU trainer!!
[LightGBM] [Info] Total Bins 2017
[LightGBM] [Info] Number of data points in the train set: 26206, number of used features: 14
[LightGBM] [Info] Using GPU Device: Tesla T4, Vendor: NVIDIA Corporation
[LightGBM] [Info] Compiling OpenCL Kernel with 256 bins...
[LightGBM] [Info] GPU programs have been built
[LightGBM] [Info] Size of histogram bin entry: 8
[LightGBM] [Info] 11 dense feature groups (0.30 MB) transferred to GPU in 0.000925 secs. 1 sparse feature groups
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.335152 -> initscore=-0.684973
[LightGBM] [Info] Start training from score -0.684973


/usr/local/lib/python3.11/dist-packages/lightgbm/basic.py:2118: UserWarning: categorical_feature keyword has been found in `params` and will be ignored.
Please use categorical_feature argument of the Dataset constructor to pass this parameter.
  _log_warning(
/usr/local/lib/python3.11/dist-packages/lightgbm/basic.py:2140: UserWarning: categorical_feature in param dict is overridden.
  _log_warning(f"{cat_alias} in param dict is overridden.")


GridSearchCV(cv=StratifiedKFold(n_splits=10, random_state=42, shuffle=True),
             estimator=LGBMClassifier(categorical_feature=['lang'],
                                      colsample_bytree=0.8, device='gpu',
                                      metric='auc', n_estimators=500, n_jobs=-1,
                                      objective='binary', random_state=42,
                                      scale_pos_weight=1.9837745516652434,
                                      subsample=0.8),
             n_jobs=1,
             param_grid={'learning_rate': [0.001, 0.01, 0.02],
                         'n_estimators': [400, 500, 600],
                         'num_leaves': [15, 30, 45]},
             scoring='roc_auc', verbose=2)

In [136]:
best_lgb_model = grid_search_lgb.best_estimator_
train_target_proba = best_lgb_model.predict_proba(train_predictors)[:, 1]
train_auc_lgb = metrics.roc_auc_score(train_target, train_target_proba)
print(f"Train AUC for tuned XGBoost: {train_auc_lgb:.4f}")

Train AUC for tuned XGBoost: 0.9756


In [137]:
test_set_predictors = test_set.drop(columns=['index'])
test_set_target_proba = best_lgb_model.predict_proba(test_set_predictors)[:, 1]

test_set_prob = pd.DataFrame({'index': test_set['index'], 'target': test_set_target_proba})
test_set_prob.to_csv('/content/drive/MyDrive/10 Github/CS610_AML/01 Assignment 1/lgb_test_set_prob_3.csv', index=False)